In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
%cd /Users/cgu3/Documents/conditional-graph-variational-autoencoder
%pwd

/Users/cgu3/Documents/conditional-graph-variational-autoencoder


'/Users/cgu3/Documents/conditional-graph-variational-autoencoder'

In [92]:
# Read the JSON file line by line into a DataFrame
data = []
with open('results/results.json', 'r') as f:
    for line in f:
        data.append(pd.read_json(line, lines=True))

# Concatenate all the dataframes
df = pd.concat(data, ignore_index=True)

# Convert the DataFrame to a CSV file
# df.to_csv('results.csv', index=False)

/var/folders/xs/_jggb8_n03q3bxbybznls2w1q5_5xx/T/ipykernel_79586/513552661.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data.append(pd.read_json(line, lines=True))
/var/folders/xs/_jggb8_n03q3bxbybznls2w1q5_5xx/T/ipykernel_79586/513552661.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data.append(pd.read_json(line, lines=True))
/var/folders/xs/_jggb8_n03q3bxbybznls2w1q5_5xx/T/ipykernel_79586/513552661.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data.append(pd.read_json(line, lines=True))
/var/folders/xs/_jggb8_n03q3bxbybznls2w1q5_5xx/T/ipykernel_79586/513552661.py:5: FutureWarning: Passing literal 

In [93]:
df

,dataset,split_ratio,seed,AUC,AP,execution_time
0,Cora,0.1,42,0.7438,0.8278,1.22
1,Cora,0.1,43,0.8595,0.8836,1.27
2,Cora,0.1,44,0.8182,0.8880,1.51
3,Cora,0.1,45,0.9339,0.9287,1.51
4,Cora,0.1,46,0.8843,0.8977,2.51
...,...,...,...,...,...,...
175,PubMed,0.9,47,0.9348,0.9382,25.76
176,PubMed,0.9,48,0.9462,0.9463,24.43
177,PubMed,0.9,49,0.9458,0.9478,26.27
178,PubMed,0.9,50,0.9386,0.9433,21.06


In [94]:
# calculate mean and std for auc and ap group by dataset and split_ratio
df_agg = df.groupby(['dataset', 'split_ratio', 'regularization']).agg({'AUC': ['mean', 'std'], 'AP': ['mean', 'std']})
# ungroup the index
df_agg = df_agg.reset_index()

In [ ]:
df_agg[df_agg['dataset'] == 'Cora']

In [ ]:
# plot lineplot for AUC and AP for each dataset and split_ratio
fig, ax = plt.subplots(2, 1, figsize=(10, 10))
ax[0].plot(df_agg[df_agg['dataset'] == 'Cora']['split_ratio'], 
           df_agg[df_agg['dataset'] == 'Cora']['AUC']['mean'], label='Cora')
# add error bar for AUC
ax[0].errorbar(df_agg[df_agg['dataset'] == 'Cora']['split_ratio'], 
               df_agg[df_agg['dataset'] == 'Cora']['AUC']['mean'], 
               yerr=df_agg[df_agg['dataset'] == 'Cora']['AUC']['std'], fmt='o')
# axist 0 plot for CiteSeer
ax[0].plot(df_agg[df_agg['dataset'] == 'CiteSeer']['split_ratio'], 
           df_agg[df_agg['dataset'] == 'CiteSeer']['AUC']['mean'], label='CiteSeer')
# add error bar for AUC
ax[0].errorbar(df_agg[df_agg['dataset'] == 'CiteSeer']['split_ratio'], 
               df_agg[df_agg['dataset'] == 'CiteSeer']['AUC']['mean'], 
               yerr=df_agg[df_agg['dataset'] == 'CiteSeer']['AUC']['std'], fmt='o')
# add legend
ax[0].legend()
# axis 1 plot for AP
ax[1].plot(df_agg[df_agg['dataset'] == 'Cora']['split_ratio'], 
           df_agg[df_agg['dataset'] == 'Cora']['AP']['mean'], label='Cora')
# add error bar for AP
ax[1].errorbar(df_agg[df_agg['dataset'] == 'Cora']['split_ratio'], 
               df_agg[df_agg['dataset'] == 'Cora']['AP']['mean'], 
               yerr=df_agg[df_agg['dataset'] == 'Cora']['AP']['std'], fmt='o')
# axist 1 plot for CiteSeer
ax[1].plot(df_agg[df_agg['dataset'] == 'CiteSeer']['split_ratio'], 
           df_agg[df_agg['dataset'] == 'CiteSeer']['AP']['mean'], label='CiteSeer')
# add error bar for AP
ax[1].errorbar(df_agg[df_agg['dataset'] == 'CiteSeer']['split_ratio'], 
               df_agg[df_agg['dataset'] == 'CiteSeer']['AP']['mean'], 
               yerr=df_agg[df_agg['dataset'] == 'CiteSeer']['AP']['std'], fmt='o')
# axist 1 plot for PubMed
ax[1].plot(df_agg[df_agg['dataset'] == 'PubMed']['split_ratio'],
           df_agg[df_agg['dataset'] == 'PubMed']['AP']['mean'], label='PubMed')
# add error bar for AP
ax[1].errorbar(df_agg[df_agg['dataset'] == 'PubMed']['split_ratio'],
               df_agg[df_agg['dataset'] == 'PubMed']['AP']['mean'],
               yerr=df_agg[df_agg['dataset'] == 'PubMed']['AP']['std'], fmt='o')
# add legend
ax[1].legend()

In [98]:
# Read the JSON file line by line into a DataFrame
data = []
with open('results/vgae_results.json', 'r') as f:
    for line in f:
        data.append(pd.read_json(line, lines=True))

# Concatenate all the dataframes
df_vgae = pd.concat(data, ignore_index=True)


/var/folders/xs/_jggb8_n03q3bxbybznls2w1q5_5xx/T/ipykernel_79586/898799659.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data.append(pd.read_json(line, lines=True))
/var/folders/xs/_jggb8_n03q3bxbybznls2w1q5_5xx/T/ipykernel_79586/898799659.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data.append(pd.read_json(line, lines=True))
/var/folders/xs/_jggb8_n03q3bxbybznls2w1q5_5xx/T/ipykernel_79586/898799659.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data.append(pd.read_json(line, lines=True))
/var/folders/xs/_jggb8_n03q3bxbybznls2w1q5_5xx/T/ipykernel_79586/898799659.py:5: FutureWarning: Passing literal 

In [99]:
# calculate mean and std for auc and ap group by dataset
df_vgae.groupby(['dataset']).agg({'AUC': ['mean', 'std'], 'AP': ['mean', 'std']})

AUC                  AP          
              mean       std      mean       std
dataset                                         
CiteSeer  0.897250  0.016827  0.893160  0.017933
Cora      0.880982  0.011835  0.878855  0.012996
PubMed    0.903411  0.009961  0.899289  0.011446